# Slowed and Reverb Music Video Maker

This notebook works best with [YouTube Music](music.youtube.com) (due to the auto naming feature when generating S&R videos at last step). Alternatively, the original [YouTube website](youtube.com) works for this code.

Having issues with this notebook? Reach out to me [here](https://github.com/asherchok/snr/issues).

### Specify the PATH for the videos to be saved here

In [ ]:
print(
"""
Insert file directory to save videos down here without \\ at the end. 
For example, C:\\Users\\user\\Downloads instead of C:\\Users\\user\\Downloads\\.
"""
)
PATH = input("> ")

### Run this block to get mp3 file and modify speed & reverb of the mp3

In [ ]:
#library for (1) downloading youtube videos
from __future__ import unicode_literals
import youtube_dl

#library for (2) audio modification & ipywidgets
import glob
import os
import time

from ipywidgets import interact
import ipywidgets as widgets
from pedalboard.io import AudioFile
from IPython.display import Audio 
from pedalboard import Pedalboard, Reverb, Resample
from IPython.display import display


###(1) code block for youtube videos
print('Paste youtube URL?')
url_choice = input("> ")

tempfileloc = PATH + '\\temp\\%(title)s.%(ext)s'
tempfileloc_wav = PATH + '\\temp\\processed-output.wav' #for (2)

ydl_opts = {
    'outtmpl': tempfileloc ,
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',
    }],
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url_choice])
    

###(2) code block for audio modification & ipywidgets
mostrecent = PATH + '\\temp\\*'
recentfile = glob.glob(mostrecent)
audioname = max(recentfile, key=os.path.getctime) #set most recent generated mp3 in /temp/ to wanted audio file

def audiomodifier(speed, reverb_rate):
    global updated_mp3_path, audioname
    
    output = widgets.Output()
    display(output)
    with output:
        print("Loading... Please wait.")
    with AudioFile(audioname) as f:
        audio = f.read(f.frames)
        samplerate = f.samplerate

    pedals = []
    if reverb_rate != 0:
        pedals.append(Reverb(room_size=reverb_rate))
    # add the Resample pedal to the list of pedals, specifying the target sample rate and quality
    pedals.append(Resample(target_sample_rate=int(samplerate * speed), quality=Resample.Quality.WindowedSinc))
    
    board = Pedalboard(pedals)
    effected = board(audio, samplerate)
    
    #show original downloaded mp3 if original settings, else show modified ones:
    if speed == 1.0 and reverb_rate == 0.0:
        display(Audio(data=audio, rate=samplerate, autoplay=True))
    else:
        samplerate2 = int(samplerate * speed)
        display(Audio(data=effected, rate=samplerate2, autoplay=True))
        with AudioFile(tempfileloc_wav, 'w', samplerate2, effected.shape[0]) as f:
            f.write(effected)
    output.clear_output()
    return speed, reverb_rate

print("""
Adjust the slow & reverb of the song with the slider! Usually speed = 0.85 & reverb = 0.10 sounds really nice for S&R songs.

It takes like around a minute to process the song everytime the slider gets adjusted.

Like what you're listening? Proceed to the next block to select GIF.
\n
""")
interact(audiomodifier, speed=widgets.FloatSlider(min=0.1, max=3.0, step=0.05, value=0.85), 
                reverb_rate=widgets.FloatSlider(min=0., max=1.0, step=0.05, value=0.1), delay=4)

### Select a GIF to loop for the entire duration of the video.

In [ ]:
from ipyfilechooser import FileChooser
fc = FileChooser()
# Can put PATH into FileChooser argument if GIFs are saved in the same directory as the .mp4 files
display(fc)

###  (Optional) Confirm your selection of your GIF by running this block

In [ ]:
print('You have selected: \n')
print(fc.selected_filename)

file = open(selected_gif_path, "rb")
image = file.read()
widgets.Image(
    value=image,
    format='gif',
)

## Code block for fonts in video

Font is set to Brush-Script-MT-Italic for the fancy font by default. Need alternative fonts other than the default? Select other fonts with the drop down selection box.

Note that some fonts might not display properly on notebook.

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets

#read in the text file with the list of fonts
with open("textclipfonts.txt", "r") as f:
    fonts = [line.strip() for line in f]

#create a dropdown widget with the list of fonts
dropdown = widgets.Dropdown(options=fonts, value='Brush-Script-MT-Italic')

#default font
font= 'Brush-Script-MT-Italic'

#function to update the sample text when a new font is selected
def update_font(change):
    global font
    font = change['new']  # Get the new font from the dropdown widget
    # Set the font of the sample text to the selected font
    sample_text.value = f"<span style='font-family:{font}; font-size:30px;'>Selected font</span>"

#attach the update function to the dropdown widget's `observe` event
dropdown.observe(update_font, names='value')

#create a text widget to display the sample text
sample_text = widgets.HTML(value="Selected font")

#display the dropdown and sample text widgets
display(dropdown)
display(sample_text)

# Run this block to get a Slowed & Reverb video!

In [ ]:
import moviepy.editor as mp
from moviepy.editor import ColorClip,TextClip,AudioFileClip
import os #for video file name
import shutil #for removing temp folder

#audio
audioclip = AudioFileClip(tempfileloc_wav)

#dark background
hdres = [1280, 720]
black_clip = ColorClip(size = hdres, color = [0,0,0]).set_duration(audioclip.duration)

#selected GIF
selected_gif_path = fc.selected
animated_gif = (mp.VideoFileClip(selected_gif_path)
          .resize(width=int(0.5 * black_clip.size[0]), height=int(0.5 * black_clip.size[1]))
          .loop()
          .set_duration(audioclip.duration)
          .set_pos(("center","center")))

#custom made formula to set words below the animated GIF
var_y = 0.5 * (black_clip.size[1] - animated_gif.size[1])
new_y = animated_gif.size[1] + 1.25 * var_y
new_x = "center" 

#audio file rename so that file gets saved outside of temp folder
filename_abbrv = (os.path.splitext(recentfile[0])[0]).replace("\\temp", "")
#Title
#file_name, file_ext = os.path.splitext(filename_abbrv)
#titlename = file_name
name_caption = filename_abbrv.replace(PATH + '\\', "")  + ' (slowed & Reverbed)'
#name_caption = 'Artist - Song Name'  + ' (slowed & Reverbed)'
title_clip = TextClip(txt=name_caption, fontsize=30, font=font, color='white')
title = title_clip.set_pos((new_x,new_y)).set_duration(audioclip.duration)

setaudioclip = animated_gif.set_audio(audioclip)

finalfilename = filename_abbrv + ' (Slowed & Reverb).mp4'
final = mp.CompositeVideoClip([black_clip, setaudioclip, title])
final.write_videofile(finalfilename)

## delete temp folder line
deletetempfolder = PATH + '\\temp'
shutil.rmtree(deletetempfolder)